In [1]:
import torch
import pyterrier as pt

if not pt.started():
    pt.init(tqdm="notebook")

PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


True

In [2]:
dataset = pt.get_dataset("irds:beir/arguana")

In [3]:
bm25 = pt.BatchRetrieve("arguana", wmodel="BM25")

In [4]:
candidates = (~bm25 % 5)(dataset.get_topics())
candidates

,qid,docid,docno,rank,score,query
0,test-environment-aeghhgwpe-pro02a,1,test-environment-aeghhgwpe-pro02a,0,162.108333,being vegetarian helps the environment becomin...
1,test-environment-aeghhgwpe-pro02a,0,test-environment-aeghhgwpe-pro02b,1,66.179093,being vegetarian helps the environment becomin...
2,test-environment-aeghhgwpe-pro02a,7,test-environment-aeghhgwpe-pro04a,2,43.521178,being vegetarian helps the environment becomin...
3,test-environment-aeghhgwpe-pro02a,2,test-environment-aeghhgwpe-pro03b,3,39.350852,being vegetarian helps the environment becomin...
4,test-environment-aeghhgwpe-pro02a,1021,test-international-sepiahbaaw-pr,4,38.703145,being vegetarian helps the environment becomin...
...,...,...,...,...,...,...
1405000,test-society-epsihbdns-con01a,656,test-economy-epsihbdns-con01a,0,72.172524,freedom of movement is an intrinsic human righ...
1405001,test-society-epsihbdns-con01a,3561,training-free-speech-debate-ifdf,1,28.272793,freedom of movement is an intrinsic human righ...
1405002,test-society-epsihbdns-con01a,5505,training-philosophy-eppprfmhb-pr,2,26.358245,freedom of movement is an intrinsic human righ...
1405003,test-society-epsihbdns-con01a,4603,training-international-glhrsimhw,3,25.718948,freedom of movement is an intrinsic human righ...


### Artic-Embed

In [5]:
from fast_forward.encoder import TransformerEncoder
import torch

class SnowFlakeDocumentEncoder(TransformerEncoder):
  def __call__(self, texts):
    document_tokens =  self.tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512)
    document_tokens.to(self.device)
    # Compute token embeddings
    with torch.no_grad():
        #query_embeddings = self.model(**query_tokens)[0][:, 0]
        doument_embeddings = self.model(**document_tokens)[0][:, 0]

    # normalize embeddings
    #query_embeddings = torch.nn.functional.normalize(query_embeddings, p=2, dim=1)
    doument_embeddings = torch.nn.functional.normalize(doument_embeddings, p=2, dim=1)
    return doument_embeddings.detach().cpu().numpy()

In [6]:
class SnowFlakeQueryEncoder(TransformerEncoder):
  def __call__(self, texts):
    query_prefix = 'Represent this sentence for searching relevant passages: '
    queries_with_prefix = ["{}{}".format(query_prefix, i) for i in texts]
    query_tokens = self.tokenizer(queries_with_prefix, padding=True, truncation=True, return_tensors='pt', max_length=512)

    query_tokens.to(self.device)
    #document_tokens =  self.tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512)
    # Compute token embeddings
    with torch.no_grad():
        query_embeddings = self.model(**query_tokens)[0][:, 0]
        #doument_embeddings = self.model(**document_tokens)[0][:, 0]

    # normalize embeddings
    query_embeddings = torch.nn.functional.normalize(query_embeddings, p=2, dim=1)
    #doument_embeddings = torch.nn.functional.normalize(doument_embeddings, p=2, dim=1)
    return query_embeddings.detach().cpu().numpy()

In [7]:
doc_encoder_artic = SnowFlakeDocumentEncoder('Snowflake/snowflake-arctic-embed-m', device="cuda:0")
q_encoder_artic = SnowFlakeQueryEncoder('Snowflake/snowflake-arctic-embed-m')

/home/dodo/.local/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at Snowflake/snowflake-arctic-embed-m and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/dodo/.local/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at Snowflake/snowflake-arctic-embed-m and are newly i

In [8]:
from fast_forward import OnDiskIndex, Mode
from pathlib import Path

ff_index_artic = OnDiskIndex.load(
    Path("ffindex_arguana_snowflake.h5"), query_encoder=q_encoder_artic, mode=Mode.MAXP
)

100%|██████████| 8674/8674 [00:00<00:00, 1108918.34it/s]


In [9]:
ff_index_artic = ff_index_artic.to_memory()

In [10]:
from fast_forward.util.pyterrier import FFScore

ff_score_artic = FFScore(ff_index_artic)

In [17]:
re_ranked = ff_score_artic(candidates) # time it - do it separately from the experiments
re_ranked

no vectors for test-international-sepiahbaaw-pr
no vectors for training-environment-aelmmhbahr-
no vectors for training-environment-assghbansb-
no vectors for training-environment-achbessbp-c
no vectors for training-environment-ahwbsawhnbs
no vectors for validation-environment-rahwbuaos
no vectors for test-international-iiahwagit-con
no vectors for validation-international-apwhber
no vectors for test-international-iiahwagit-pro
no vectors for training-international-aghwaucsa
no vectors for validation-education-uehwsnwu-pr
no vectors for validation-education-uehwsnwu-co
no vectors for training-environment-ogecephwgn-
no vectors for training-environment-cpiahwdwf-p
no vectors for training-environment-crasbmurcc-
no vectors for training-environment-cehwirtrnp-
no vectors for training-environment-echbcatspct
no vectors for test-environment-opecewiahw-pro0
no vectors for test-environment-opecewiahw-con0
no vectors for training-environment-eegwiahbe-p
no vectors for training-environment-cegw

,qid,docno,score_0,score,query
0,test-environment-aeghhgwpe-pro02a,test-environment-aeghhgwpe-pro02a,162.108333,0.926701,being vegetarian helps the environment becomin...
1,test-environment-aeghhgwpe-pro02a,test-environment-aeghhgwpe-pro02b,66.179093,0.738024,being vegetarian helps the environment becomin...
2,test-environment-aeghhgwpe-pro02a,test-environment-aeghhgwpe-pro04a,43.521178,0.493260,being vegetarian helps the environment becomin...
3,test-environment-aeghhgwpe-pro02a,test-environment-aeghhgwpe-pro03b,39.350852,0.552747,being vegetarian helps the environment becomin...
4,test-environment-aeghhgwpe-pro01a,test-environment-aeghhgwpe-pro01a,106.032703,0.871509,it is immoral to kill animals as evolved human...
...,...,...,...,...,...
4558,test-society-epsihbdns-con03a,test-economy-epsihbdns-pro03a,87.009874,0.544814,rural life is miserable and has higher mortali...
4559,test-society-epsihbdns-con03a,test-economy-epsihbdns-pro02a,77.921501,0.455330,rural life is miserable and has higher mortali...
4560,test-society-epsihbdns-con03a,test-economy-epsihbdns-con02b,66.800262,0.435975,rural life is miserable and has higher mortali...
4561,test-society-epsihbdns-con03a,test-society-epsihbdns-con02b,66.800262,0.435975,rural life is miserable and has higher mortali...


In [12]:
from fast_forward.util.pyterrier import FFInterpolate

alpha_values = [0.05, 0.1, 0.3, 0.5, 0.8]
dataframes = []

## Trying to use different alpha values 
for alpha in alpha_values:
    ff_int = FFInterpolate(alpha=alpha)
    dataframes.append(ff_int(re_ranked))

In [13]:
ff_int2 = FFInterpolate(alpha=0.1)
ff_int2(re_ranked)

,qid,docno,query,score
0,test-environment-aeghhgwpe-pro02a,test-environment-aeghhgwpe-pro02a,being vegetarian helps the environment becomin...,17.044864
1,test-environment-aeghhgwpe-pro02a,test-environment-aeghhgwpe-pro02b,being vegetarian helps the environment becomin...,7.282131
2,test-environment-aeghhgwpe-pro02a,test-environment-aeghhgwpe-pro04a,being vegetarian helps the environment becomin...,4.796052
3,test-environment-aeghhgwpe-pro02a,test-environment-aeghhgwpe-pro03b,being vegetarian helps the environment becomin...,4.432558
4,test-environment-aeghhgwpe-pro01a,test-environment-aeghhgwpe-pro01a,it is immoral to kill animals as evolved human...,11.387628
...,...,...,...,...
4558,test-society-epsihbdns-con03a,test-economy-epsihbdns-pro03a,rural life is miserable and has higher mortali...,9.191320
4559,test-society-epsihbdns-con03a,test-economy-epsihbdns-pro02a,rural life is miserable and has higher mortali...,8.201948
4560,test-society-epsihbdns-con03a,test-economy-epsihbdns-con02b,rural life is miserable and has higher mortali...,7.072404
4561,test-society-epsihbdns-con03a,test-society-epsihbdns-con02b,rural life is miserable and has higher mortali...,7.072404


In [18]:
dataframes[1] ## This is the dataframe for alpha=0.1 - same as the one above

,qid,docno,query,score
0,test-environment-aeghhgwpe-pro02a,test-environment-aeghhgwpe-pro02a,being vegetarian helps the environment becomin...,17.044864
1,test-environment-aeghhgwpe-pro02a,test-environment-aeghhgwpe-pro02b,being vegetarian helps the environment becomin...,7.282131
2,test-environment-aeghhgwpe-pro02a,test-environment-aeghhgwpe-pro04a,being vegetarian helps the environment becomin...,4.796052
3,test-environment-aeghhgwpe-pro02a,test-environment-aeghhgwpe-pro03b,being vegetarian helps the environment becomin...,4.432558
4,test-environment-aeghhgwpe-pro01a,test-environment-aeghhgwpe-pro01a,it is immoral to kill animals as evolved human...,11.387628
...,...,...,...,...
4558,test-society-epsihbdns-con03a,test-economy-epsihbdns-pro03a,rural life is miserable and has higher mortali...,9.191320
4559,test-society-epsihbdns-con03a,test-economy-epsihbdns-pro02a,rural life is miserable and has higher mortali...,8.201948
4560,test-society-epsihbdns-con03a,test-economy-epsihbdns-con02b,rural life is miserable and has higher mortali...,7.072404
4561,test-society-epsihbdns-con03a,test-society-epsihbdns-con02b,rural life is miserable and has higher mortali...,7.072404


In [15]:
from pyterrier.measures import RR, nDCG, MAP

pt.Experiment(
    [~bm25 % 1000 >> ff_score_artic >> dataframes[1]],
    dataset.get_topics(),
    dataset.get_qrels(),
    eval_metrics=[RR @ 10, nDCG @ 10, MAP @ 100],
    names=["BM25 >> FF"],
  )

/tmp/ipykernel_19230/1759268779.py:4: DeprecationWarning: Coercion of a dataframe into a transformer is deprecated; use a pt.Transformer.from_df() instead
  [~bm25 % 1000 >> ff_score_artic >> dataframes[1]],
no vectors for test-international-sepiahbaaw-pr
no vectors for training-environment-ahwbsawhnbs
no vectors for training-environment-ogecephwgn-
no vectors for validation-environment-rahwbuaos
no vectors for training-environment-raceeghg-co
no vectors for training-environment-eegwiahbe-c
no vectors for training-philosophy-lsfhwnhc-pro
no vectors for training-environment-cephbesane-
no vectors for training-environment-cegwhwbhd-p
no vectors for training-environment-cephwcear-p
no vectors for training-environment-assghbansb-
no vectors for training-environment-pscisghwbs3
no vectors for training-health-hgwhwbutffs-con0
no vectors for training-society-gfyhbprcsao-con
no vectors for training-economy-epiahbnhba-con0
no vectors for training-environment-echbcatspct
no vectors for training-

,name,RR@10,nDCG@10,AP@100
0,BM25 >> FF,0.163751,0.222852,0.163751


In [16]:
pt.Experiment(
    [~bm25 % 1000 >> ff_score_artic >> ff_int2],
    dataset.get_topics(),
    dataset.get_qrels(),
    eval_metrics=[RR @ 10, nDCG @ 10, MAP @ 100],
    names=["BM25 >> FF"],
  )

no vectors for test-international-sepiahbaaw-pr
no vectors for training-environment-ahwbsawhnbs
no vectors for training-environment-ogecephwgn-
no vectors for validation-environment-rahwbuaos
no vectors for training-environment-raceeghg-co
no vectors for training-environment-eegwiahbe-c
no vectors for training-philosophy-lsfhwnhc-pro
no vectors for training-environment-cephbesane-
no vectors for training-environment-cegwhwbhd-p
no vectors for training-environment-cephwcear-p
no vectors for training-environment-assghbansb-
no vectors for training-environment-pscisghwbs3
no vectors for training-health-hgwhwbutffs-con0
no vectors for training-society-gfyhbprcsao-con
no vectors for training-economy-epiahbnhba-con0
no vectors for training-environment-echbcatspct
no vectors for training-environment-cegwhwbhd-c
no vectors for training-international-ehbbsgier
no vectors for training-economy-beghwprsci-con0
no vectors for training-economy-epiaghwpto-pro0
no vectors for training-environment-ceph

,name,RR@10,nDCG@10,AP@100
0,BM25 >> FF,0.195528,0.287237,0.203235
